In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import dateutil.easter as easter
import holidays
import datetime 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# !pip install flaml
# from flaml import AutoML

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold
%matplotlib inline

### Loading Data

In [2]:
train_df = pd.read_csv('../input/tabular-playground-series-sep-2022/train.csv')
train_df.date= pd.to_datetime(train_df.date)
train_df.head()

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240


In [3]:
test_df = pd.read_csv('../input/tabular-playground-series-sep-2022/test.csv')
test_df.date= pd.to_datetime(train_df.date)
test_df.head()

,row_id,date,country,store,product
0,70128,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,70129,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,70130,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,70131,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,70132,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques


### Cleaning

In [4]:
# Null test

train_df.isnull().sum()

row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [5]:
# Duplicates

train_df.duplicated().unique()

array([False])

In [6]:
# Dates with higher importance on sale
important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,16,17, 124, 125, 126, 127, 140, 141,142, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]

### Feature Engineering

Feature Engineering has been taken from [here](https://www.kaggle.com/code/nanduvardhanreddy/tbh-sep-2022).

In [7]:
# monthly and weekly grouping
weekly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="W")])["num_sold"].sum().rename("num_sold").reset_index()
monthly_df = train_df.groupby(["country","store", "product", pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()

product_store_weights = monthly_df.groupby(["product","store"])["num_sold"].sum() / monthly_df.groupby(["product"])["num_sold"].sum()

store_weights = train_df.groupby("store")["num_sold"].sum()/train_df["num_sold"].sum()

new_monthly_df = monthly_df.loc[monthly_df["date"] < "2020-01-01"]
product_country_weights = new_monthly_df.groupby(["product","country"])["num_sold"].sum() / new_monthly_df.groupby(["product"])["num_sold"].sum()

product_country_weights.reset_index().groupby("country")["num_sold"].mean().loc["France"] / product_country_weights.reset_index().groupby("country")["num_sold"].mean()

product_df = train_df.groupby(["date","product"])["num_sold"].sum().reset_index()

product_ratio_df = product_df.pivot(index="date", columns="product", values="num_sold")
product_ratio_df = product_ratio_df.apply(lambda x: x/x.sum(),axis=1)
product_ratio_df = product_ratio_df.stack().rename("ratios").reset_index()

train_df = train_df.groupby(["date"])["num_sold"].sum().reset_index()
weekly_df = train_df.groupby([pd.Grouper(key="date", freq="W")])["num_sold"].sum().rename("num_sold").reset_index()
monthly_df = train_df.groupby([pd.Grouper(key="date", freq="MS")])["num_sold"].sum().rename("num_sold").reset_index()

# Cleaning data post Covid outbreak

train_df = train_df.loc[~((train_df["date"] >= "2020-03-01") & (train_df["date"] < "2020-06-01"))]

# Forecast df
test_all_df = test_df.groupby(["date"])["row_id"].first().reset_index().drop(columns="row_id")
# Dates to forecast
test_all_df_dates = test_all_df[["date"]]

In [8]:
def feature_engineer(df):
    new_df = df.copy()
    new_df["month"] = df["date"].dt.month
    new_df["month_sin"] = np.sin(new_df['month'] * (2 * np.pi / 12))
    
    new_df["day"] = df["date"].dt.day
    new_df["day_sin"] = np.sin(new_df['day'] * (2 * np.pi / 12))
    
    new_df["day_of_week"] = df["date"].dt.dayofweek
    new_df["day_of_week"] = new_df["day_of_week"].apply(lambda x: 0 if x<=3 else(1 if x==4 else (2 if x==5 else (3))))
    
    new_df["day_of_year"] = df["date"].dt.dayofyear

    new_df["day_of_year"] = new_df.apply(lambda x: x["day_of_year"]-1 if (x["date"] > pd.Timestamp("2020-02-29") and x["date"] < pd.Timestamp("2021-01-01"))  else x["day_of_year"], axis=1)
    
    important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 124, 125, 126, 127, 140, 141, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 
                       180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]
    

    new_df["important_dates"] = new_df["day_of_year"].apply(lambda x: x if x in important_dates else 0)
    
    
    new_df["year"] = df["date"].dt.year
    
    easter_date = new_df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    for day in list(range(-5, 5)) + list(range(40, 48)):
        new_df[f'easter_{day}'] = (new_df.date - easter_date).dt.days.eq(day)
    new_df = new_df.drop(columns=["date","month","day", "day_of_year"])
    
    for col in new_df.columns :
        if 'easter' in col :
            new_df = pd.get_dummies(new_df, columns = [col], drop_first=True)
    
    new_df = pd.get_dummies(new_df, columns = ["important_dates","day_of_week"], drop_first=True)
    
    return new_df

In [9]:
def get_holidays(df):
    years_list = [2017, 2018, 2019, 2020, 2021]

    holiday_BE = holidays.CountryHoliday('BE', years = years_list)
    holiday_FR = holidays.CountryHoliday('FR', years = years_list)
    holiday_DE = holidays.CountryHoliday('DE', years = years_list)
    holiday_IT = holidays.CountryHoliday('IT', years = years_list)
    holiday_PL = holidays.CountryHoliday('PL', years = years_list)
    holiday_ES = holidays.CountryHoliday('ES', years = years_list)

    holiday_dict = holiday_BE.copy()
    holiday_dict.update(holiday_FR)
    holiday_dict.update(holiday_DE)
    holiday_dict.update(holiday_IT)
    holiday_dict.update(holiday_PL)
    holiday_dict.update(holiday_ES)

    df['holiday_name'] = df['date'].map(holiday_dict)
    df['is_holiday'] = np.where(df['holiday_name'].notnull(), 1, 0)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
    
    return df



def encode_holiday_names(df, enc, subset='train'):
    if subset=='train':
        df['holiday_name'] = enc.fit_transform(df['holiday_name'].values.reshape(-1,1))
    else:
        df['holiday_name'] = enc.transform(df['holiday_name'].values.reshape(-1,1))
        not_hol_val = oe.transform([['Not Holiday']])[0,0]
        df.loc[df['holiday_name']==-1, 'holiday_name'] = not_hol_val
    return df

In [10]:
oe = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_all_df = get_holidays(train_df)
test_all_df = get_holidays(test_all_df)
train_all_df = feature_engineer(train_all_df)
test_all_df = feature_engineer(test_all_df)
train_all_df = encode_holiday_names(train_all_df, oe)
test_all_df = encode_holiday_names(test_all_df, oe)

### Training

In [11]:
y = train_all_df["num_sold"]
X = train_all_df.drop(columns="num_sold")
X_test = test_all_df


X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=100, test_size=0.25)

In [12]:
# from flaml import AutoML
# # Setting up automl regression problem

# # Training
# automl = AutoML()

# # Constraints 
# automl_settings = {
#     "time_budget": 4000,
#     "metric": 'mape',
#     "task": 'regression',
#     #"estimator_list" : ["lgbm"],
#     "log_file_name": "TPS_Sep22 AutoML.log"
# }

# automl.fit(X_train=X_train, y_train=y_train, **automl_settings)
# print(automl)

In [13]:
# Competition metric

def smape(y_true, y_pred):
    smape = abs(y_true - y_pred) / (abs(y_true) + abs(y_pred))
    smape = smape.mean() * 200
    return smape

In [14]:
# Ridge

ridge_preds = np.zeros(X_test.shape[0])
avg_smape = 0
n=0

kf = GroupKFold(n_splits=4)

for trn_idx, test_idx in kf.split(X, groups=X.year) :
    x_train, x_valid = X.iloc[trn_idx], X.iloc[test_idx]
    y_train, y_valid = y.iloc[trn_idx], y.iloc[test_idx]
    
    ridge_model = Ridge(alpha=0.2, tol=0.00001, max_iter=10000)
    #ridge_model = make_pipeline(preprocessing.StandardScaler(), ridge_model)
    ridge_model.fit(x_train, y_train)

    y_pred = ridge_model.predict(x_valid)
    avg_smape += smape(y_pred, y_valid)

    n = n + 1

    test_pred = ridge_model.predict(X_test)
    pred = pd.Series(test_pred) 

    ridge_preds += pred / kf.n_splits

print(f"smape: {avg_smape/kf.n_splits}")

smape: 9.87453088150577


In [15]:
# Linear Regression

linear_preds = np.zeros(X_test.shape[0])
avg_smape = 0
n=0

kf = GroupKFold(n_splits=4)

for trn_idx, test_idx in kf.split(X, groups=X.year) :
    x_train, x_valid = X.iloc[trn_idx], X.iloc[test_idx]
    y_train, y_valid = y.iloc[trn_idx], y.iloc[test_idx]
    
    linear_model = LinearRegression()
    linear_model = make_pipeline(preprocessing.StandardScaler(), linear_model)
    linear_model.fit(x_train, y_train)

    y_pred = ridge_model.predict(x_valid)
    avg_smape += smape(y_pred, y_valid)

    n = n + 1

    test_pred = ridge_model.predict(X_test)
    pred = pd.Series(test_pred) 

    linear_preds += pred / kf.n_splits
    
print(f"smape: {avg_smape/kf.n_splits}")

smape: 7.177064124274411


In [16]:
test_all_df_dates['num_sold'] = 0.95*linear_preds + 0.05*ridge_preds

### Prediction

In [17]:
product_ratio_2017 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2017].copy()
product_ratio_2017['mm-dd'] = product_ratio_2017['date'].dt.strftime('%m-%d')
product_ratio_2017 = product_ratio_2017.drop(columns='date')
product_ratio_2017 = product_ratio_2017.reset_index()
product_ratio_2018 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2018].copy()
product_ratio_2018['mm-dd'] = product_ratio_2018['date'].dt.strftime('%m-%d')
product_ratio_2018 = product_ratio_2018.drop(columns='date')
product_ratio_2018 = product_ratio_2018.reset_index()
product_ratio_2019 = product_ratio_df.loc[product_ratio_df['date'].dt.year == 2019].copy()
product_ratio_2019['mm-dd'] = product_ratio_2019['date'].dt.strftime('%m-%d')
product_ratio_2019 = product_ratio_2019.drop(columns='date')
product_ratio_2019 = product_ratio_2019.reset_index()
product_ratio_2019['mean_ratios'] = (product_ratio_2017['ratios']+product_ratio_2018['ratios']+product_ratio_2019['ratios'])/3


test_product_ratio_df = test_df.copy()
test_product_ratio_df['mm-dd'] = test_product_ratio_df['date'].dt.strftime('%m-%d')

test_product_ratio_df = pd.merge(test_product_ratio_df,product_ratio_2019, how="left", on = ["mm-dd","product"])

temp_df = pd.concat([product_ratio_df,test_product_ratio_df]).reset_index(drop=True)


In [18]:
original_train_df = pd.read_csv('../input/tabular-playground-series-sep-2022/train.csv', parse_dates=['date'])

In [19]:
test_sub_df = pd.merge(test_df, test_all_df_dates, how="left")
test_sub_df["ratios"] = test_product_ratio_df["ratios"]
test_sub_df["mean_ratios"] = test_product_ratio_df["mean_ratios"]

store_weights = original_train_df.groupby('store')['num_sold'].sum()/original_train_df['num_sold'].sum()

country_weights = original_train_df.groupby('country')['num_sold'].sum()/original_train_df['num_sold'].sum()

In [20]:
def disaggregate_forecast(df) :
    new_df = df.copy()
    
    store_weights = original_train_df.groupby('store')['num_sold'].sum()/original_train_df['num_sold'].sum()
    #print(store_weights)
    country_weights = pd.Series(index = test_sub_df["country"].unique(),data = 1/6)
    #print(country_weights)
    for country in country_weights.index:
        new_df.loc[(new_df["country"] == country), "num_sold"] = new_df.loc[(new_df["country"] == country), "num_sold"] *  country_weights[country]
    #print(new_df)
    for store in store_weights.index:
        new_df.loc[new_df["store"] == store, "num_sold"] = new_df.loc[new_df["store"] == store, "num_sold"] * store_weights[store]
        
    new_df["num_sold"] = new_df["num_sold"] * new_df["mean_ratios"]
    new_df["num_sold"] = new_df["num_sold"].round()
    new_df = new_df.drop(columns=["ratios"])    
    
    return new_df

In [21]:
final_df = disaggregate_forecast(test_sub_df)

### Submission

In [22]:
# Final csv 
submission_df = pd.DataFrame()
submission_df['row_id'] = pd.read_csv('../input/tabular-playground-series-sep-2022/test.csv')['row_id']
submission_df['num_sold'] =  final_df['num_sold']
submission_df.head()

,row_id,num_sold
0,70128,485.0
1,70129,396.0
2,70130,329.0
3,70131,528.0
4,70132,168.0


In [23]:
submission_df.to_csv("submission.csv", index=False)